### Initialize

In [ ]:
import requests
import pandas as pd
import os
import sys
from datetime import datetime

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))

from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path
import src.esh_client as esh

base_url = get_base_url()
root_path = get_root_path()
example_name = sys.path[0].split(os.sep)[-2]


def execute_search_query(query):
    r = requests.post(
        f'{base_url}/v1/query/{example_name}{TENANT_SUFFIX}/latest', json=[query])
    if r.status_code == 200:
        res = r.json()[0]
        if '@odata.count' in res:
            print(f'Total count: {res["@odata.count"]}')
        if res and 'value' in res and res['value']:
            df = pd.DataFrame.from_dict(res['value'])[
                ['firstName', 'lastName', 'birthDate', 'nationality']]
            df.rename(columns={'firstName': 'First Name',
                      'lastName': 'Last Name'}, inplace=True)
            display(df)
        else:
            print('No results')
    else:
        print(f'Error during search. HTTP status code {r.status_code}')
        print(r.json())


### Search for first name "Jane"

In [ ]:
so = esh.EshObject(
    count=True,
    top=1,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items= [
            esh.Comparison(
                property= esh.Property(property='firstName'),
                operator= esh.ComparisonOperator.Search,
                value= esh.StringValue(value='Jane')
            )
        ]
    )
)

execute_search_query(so.dict(exclude_none=True))

### Search for first name "Jane" and last name "Doe"

In [ ]:
so = esh.EshObject(
    count=True,
    top=1,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items= [
            esh.Comparison(
                property= esh.Property(property='firstName'),
                operator= esh.ComparisonOperator.Search,
                value= esh.StringValue(value='Jane')
            ),
            esh.Comparison(
                property= esh.Property(property='lastName'),
                operator= esh.ComparisonOperator.Search,
                value= esh.StringValue(value='Doe')
            )
        ]
    )
)

execute_search_query(so.dict(exclude_none=True))

### Exact-search for first name "June"

In [ ]:
so = esh.EshObject(
    count=True,
    top=1,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items= [
            esh.Comparison(
                property= esh.Property(property='firstName'),
                operator= esh.ComparisonOperator.Search,
                value= esh.StringValue(value='June')
            )
        ]
    )
)

execute_search_query(so.dict(exclude_none=True))

### Fuzzy-search for first name "June"

In [ ]:
so = esh.EshObject(
    count=True,
    top=1,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items= [
            esh.Comparison(
                property= esh.Property(property='firstName'),
                operator= esh.ComparisonOperator.Search,
                value= esh.StringValue(
                    value='June',
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
                )
            )
        ]
    )
)

execute_search_query(so.dict(exclude_none=True))

### Nested query

In [ ]:
so = esh.EshObject(
    count=True,
    top=10,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Expression(
                operator=esh.LogicalOperator.OR,
                items=[
                    esh.Expression(
                        operator=esh.LogicalOperator.AND,
                        items=[
                            esh.Comparison(
                                property= esh.Property(property='lastName'),
                                operator= esh.ComparisonOperator.Search,
                                value= esh.StringValue(value='Doe')),
                            esh.Comparison(
                                property= esh.Property(property='firstName'),
                                operator= esh.ComparisonOperator.Search,
                                value= esh.StringValue(value='John'))
                        ]
                    ),
                    esh.Expression(
                        operator=esh.LogicalOperator.AND,
                        items=[
                            esh.Comparison(
                                property= esh.Property(property='lastName'),
                                operator= esh.ComparisonOperator.Search,
                                value= esh.StringValue(value='Doe')),
                            esh.Comparison(
                                property= esh.Property(property='firstName'),
                                operator= esh.ComparisonOperator.Search,
                                value= esh.StringValue(value='Jane'))
                        ]
                    )
                ]
            )
        ]
    )
)

execute_search_query(so.dict(exclude_none=True))

### Freestyle-Search

In [ ]:
so = esh.EshObject(
    count=True,
    top=10,
    select=['firstName', 'lastName'],
    scope='Person',
    searchQueryFilter=esh.Expression(
        items=[
            esh.StringValue(
                value='June',
                searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7))
        ]
    )
)
execute_search_query(so.dict(exclude_none=True))